In [1]:
from secret_key import API_TOKEN, REPO_ID
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import SequentialChain, LLMChain


In [100]:

llm = HuggingFaceEndpoint(
        repo_id=REPO_ID,
        max_length=128,
        temperature=0.5,
        huggingfacehub_api_token=API_TOKEN,
    )

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\wotcw4\.cache\huggingface\token
Login successful


In [109]:
name_template = PromptTemplate(
        input_variables=["selected_cusine"],
        template="""Generate one fancy restaurant name for {selected_cusine} cuisine. Only return the name. Do not include explanations, descriptions, or any additional context.""",
    )

name_chain = LLMChain(llm = llm, prompt = name_template, output_key = "restaurant_name")
# restaurant_name = name_chain.invoke("mexican")

In [99]:
menu_item_template = PromptTemplate(
    input_variables=["selected_cusine"],
    template="""Generate 5 fancy menu items for {selected_cusine} cuisine in a clean JSON format.
    Each item should have a 'name' and 'description' field only. Do not include extra text, headings, or additional information beyond the JSON structure.""",
)

menu_chain = LLMChain(llm = llm, prompt = menu_item_template, output_key = "menu_items")
# menu = menu_chain.invoke("mexican")

In [101]:
from langchain.chains import SequentialChain
schain = SequentialChain(
    chains=[name_chain, menu_chain],
    input_variables=["selected_cusine"],
    output_variables=["restaurant_name", "menu_items"],
)
res = schain.invoke({"selected_cusine":"indian"})
print(res)

{'selected_cusine': 'indian', 'restaurant_name': '\n\n"Saffron Symphony"', 'menu_items': '\n\n    [\n        {\n            "name": "Methi Malai Mushroom",\n            "description": "Tender mushrooms cooked in a creamy fenugreek sauce, served with naan."\n        },\n        {\n            "name": "Aloo Gobi",\n            "description": "Crispy cauliflower and potatoes cooked in a fragrant spice blend, served with basmati rice."\n        },\n        {\n            "name": "Lamb Rogan Josh",\n            "description": "Tender lamb simmered in a rich, aromatic gravy made with tomatoes, yogurt, and Kashmiri spices."\n        },\n        {\n            "name": "Palak Paneer",\n            "description": "Creamy spinach puree with soft, fresh paneer cheese, served with garlic naan."\n        },\n        {\n            "name": "Biryani",\n            "description": "A flavorful rice dish layered with marinated chicken, aromatic spices, and saffron."\n        }\n    ]'}


In [105]:
import json
resname = res["restaurant_name"]
menuitem = res["menu_items"]
print(json.loads(menuitem))

[{'name': 'Methi Malai Mushroom', 'description': 'Tender mushrooms cooked in a creamy fenugreek sauce, served with naan.'}, {'name': 'Aloo Gobi', 'description': 'Crispy cauliflower and potatoes cooked in a fragrant spice blend, served with basmati rice.'}, {'name': 'Lamb Rogan Josh', 'description': 'Tender lamb simmered in a rich, aromatic gravy made with tomatoes, yogurt, and Kashmiri spices.'}, {'name': 'Palak Paneer', 'description': 'Creamy spinach puree with soft, fresh paneer cheese, served with garlic naan.'}, {'name': 'Biryani', 'description': 'A flavorful rice dish layered with marinated chicken, aromatic spices, and saffron.'}]


In [106]:
j = json.loads(menuitem)
for i in j:
    print(i["name"], i["description"])

Methi Malai Mushroom Tender mushrooms cooked in a creamy fenugreek sauce, served with naan.
Aloo Gobi Crispy cauliflower and potatoes cooked in a fragrant spice blend, served with basmati rice.
Lamb Rogan Josh Tender lamb simmered in a rich, aromatic gravy made with tomatoes, yogurt, and Kashmiri spices.
Palak Paneer Creamy spinach puree with soft, fresh paneer cheese, served with garlic naan.
Biryani A flavorful rice dish layered with marinated chicken, aromatic spices, and saffron.


In [78]:
for key, value in menu_items.items():
        menu_name = value.split('-')[0].strip()
        print(menu_name)

Quesadilla de Pollo
Tacos al Pastor
Enchiladas Rojas
Guacamole
Nachos Supremos
Chiles Rellenos
Sopa de Tortilla
Margarita


In [64]:
# import json
# # Extracting the menu_item and cleaning it
# menu_item_str = res['menu_item'].strip('```json').strip()

# # Parsing the JSON string
# menu_items = json.loads(menu_item_str)